In [3]:
%matplotlib inline
%load_ext autoreload
%autoreload 5
%autosave 15

import sklearn as sk
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import math

Autosaving every 15 seconds


In [9]:
def gradient_descent(f, df, start_point, a_start=10, a_modify=lambda a: a * 0.999, normalize=True,
                    steps=1000000, l=-1000000000, r=1000000000, min_gradient=0.00001):
    
    if (type(start_point) != int):
        p = start_point.copy()
    else:
        p = start_point
    a = a_start
    for _ in range(steps):
        gradient = df(p)
        norm = np.linalg.norm(gradient)
        if (norm < min_gradient):
            break
        if (normalize):
            gradient = gradient / norm
        p = p - a * gradient
        p = np.clip(p, l, r)
        a = a_modify(a)
    return p

def monte_carlo(f, df, count, dims, a_start=10, a_modify=lambda a: a * 0.999, normalize=True,
                steps=1000000, l=-1000000000, r=1000000000, min_gradient=0.000001):
    points = []
    values = []
    for i in range(count):
        rand_point = np.random.random(dims) * (r - l) + l
        t = gradient_descent(f, df, rand_point, a_start=a_start, a_modify=a_modify, normalize=normalize,
                            steps=steps, l=l, r=r, min_gradient=min_gradient)
        points.append(t)
        values.append(f(t))
    return points[np.argmin(values)]
    



In [30]:
f = lambda p: (1 - p[0]) ** 2 + 100 * (p[1] - p[0] ** 2) ** 2
df = lambda p: np.array([(2 * (-1 + p[0] + 200 * p[0] ** 3 - 200 * p[0] * p[1])), (-200 * (p[0] ** 2 - p[1]))])
p = np.array([100.0, -2311.0])

res = gradient_descent(f, df, p, a_start=100, a_modify=lambda a: a * 0.999, normalize=True, steps=100000)
print("2 rozenbrok gradient:",res, f(res))

2 rozenbrok gradient: [ 1.00000001  1.        ] 4.97201543071e-14


In [10]:
f = lambda p: (1 - p[0]) ** 2 + 100 * (p[1] - p[0] ** 2) ** 2
df = lambda p: np.array([(2 * (-1 + p[0] + 200 * p[0] ** 3 - 200 * p[0] * p[1])), (-200 * (p[0] ** 2 - p[1]))])
p = np.array([12000.0, -2311.0])

res = monte_carlo(f, df, 10, 2, a_start=10, a_modify=lambda a: a * 0.999, normalize=True, steps=10000,
                  l=-1000, r=1000)
print("2 rozenbrok monte carlo:", res, f(res))

2 rozenbrok monte carlo: [ 1.00000016  0.99999982] 2.50261906766e-11


In [11]:
def fn(point):
    res = 0
    for i in range(point.size - 1):
        res += (1 - point[i]) ** 2 + (point[i + 1] - point[i] ** 2) ** 2
    return res

def dfn(point, eps=0.0000001):
    res = []
    p = point.copy()
    dims = point.size
    for i in range(dims):
        p[i] += eps
        t = (fn(p) - fn(point)) / eps
        res.append(t)
        p[i] -= eps
    return res

p = np.array([12000.0, -2311.0, 4.0, 2.4, 123.2, 45.2, -123, -123, 34, 0])

res = gradient_descent(fn, dfn, p, a_start=100, a_modify=lambda a: a * 0.999, normalize=True, steps=100000)

print("10 rozenbrok gradient:",res, f(res))

10 rozenbrok gradient: [ 0.99999985  0.99999998  0.99999969  1.00000006  0.99999962  1.00000005
  0.99999961  0.99999988  0.99999944  0.99999902] 8.0219057704e-12


In [54]:

def optimal(f, df, point):
    def funcopt(a):
        grad = df(point)
        grad = grad / np.linalg.norm(grad)
        return f(point - a * grad)
    return funcopt

def doptimal(f, fd, point, eps=0.0000001):
    def funcdopt(a):
        return (optimal(f, df, point)(a + eps) - optimal(f, df, point)(a)) / eps
    return funcdopt

def gradient_descent_optimal(f, df, start_point, min_a=0.000000001, max_a=10, a_steps=100,normalize=True,
                    steps=1000000, l=-1000000000, r=1000000000, min_gradient=0.00001):
    
    if (type(start_point) != int):
        p = start_point.copy()
    else:
        p = start_point
    for _ in range(steps):
        gradient = df(p)
        norm = np.linalg.norm(gradient)
        if (norm < min_gradient):
            break
        if (normalize):
            gradient = gradient / norm
        la = min_a
        ra = max_a
        for _ in range(a_steps):
            a1 = la + (ra - la) / 3
            a2 = ra - (ra - la) / 3
            if f(p - a1 * gradient) < f(p - a2 * gradient):
                ra = a2
            else:
                la = a1
        p = p - la * gradient
        p = np.clip(p, l, r)
    return p

f = lambda p: (1 - p[0]) ** 2 + 100 * (p[1] - p[0] ** 2) ** 2
df = lambda p: np.array([(2 * (-1 + p[0] + 200 * p[0] ** 3 - 200 * p[0] * p[1])), (-200 * (p[0] ** 2 - p[1]))])
p = np.array([100.0, -2311.0])

res = gradient_descent_optimal(f, df, p, min_a=0.000000001, max_a=0.001, a_steps=20, normalize=False,
                               steps=20000, l=-1000000000, r=1000000000)
print("??:",res, f(res))

??: [ 0.99988923  0.99977802] 1.22903925219e-08
